In [1]:
pip install groq langchain langchain-core langchain-groq qdrant-client pypdf gradio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!ollama pull nomic-embed-text

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¸ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ¦ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â ‹ pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest 
pulling 970aa74c0a90... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 274 MB                         
pulling c71d239df917... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling ce4a164fc046... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   17 B                         
pulling 31df23ea7daa... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
pip install langchain-community

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
!pip install -U langchain-ollama

Defaulting to user installation because normal site-packages is not writeable


In [5]:
pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install --upgrade jupyter
!pip install --upgrade ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install protobuf==4.25.5
!pip install opentelemetry-proto grpcio-tools --upgrade

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached grpcio_tools-1.67.1-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.3 MB 4.2 MB/s eta 0:00:01
   -------------- ------------------------- 1.6/4.3 MB 3.8 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.3 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 3.4/4.3 MB 3.9 MB/s eta 0:00:01
   ------------------------------------ --- 3.9/4.3 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 3.8 MB/s eta 0:00:00
Using cached proto

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-common 1.27.0 requires opentelemetry-proto==1.27.0, but you have opentelemetry-proto 1.28.1 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.27.0 requires opentelemetry-proto==1.27.0, but you have opentelemetry-proto 1.28.1 which is incompatible.


In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_ollama import OllamaEmbeddings

In [2]:
loader = PyPDFDirectoryLoader("data")
the_text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(the_text)

vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="ollama_embeds",
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
)

retriever = vectorstore.as_retriever()

In [3]:
from langchain_groq import ChatGroq
from env import groq_api_key
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name='mixtral-8x7b-32768'
    )


#https://github.com/ollama/ollama

# from langchain_community.llms import Ollama

# llm = Ollama(model="llama2")

# llm.invoke("Tell me a joke")

In [4]:
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [5]:
import os
import time
import textwrap
import gradio as gr

# Test the architecture with a simple hard coded question
response = rag_chain.invoke("What is this document about")
print(textwrap.fill(response, width=80))

Based on the provided document metadata and page content, the document entitled
"tropicalwine.pdf" appears to be about the production of wine, specifically
fermented alcoholic beverages, from tropical fruits and the health aspects of
these beverages. The document mentions the importance of fruits as a nutritious
food and the role of fermentation in the preparation of beverages. It also
mentions the use of the yeast Saccharomyces cerevisiae in the fermentation
process. The key words listed at the beginning of the document include "Wine",
"Tropical Countries", and "Sub-Tropical Countries", as well as different types
of wine such as "Red Wine", "White Wine", and "Pink Wine". The document is
sourced from the file path "data\\tropicalwine.pdf" and is from page 1 of the
document.


In [6]:
# Make the questions dynamic using a chat interface. Let's use gradio for this.
def process_question(user_question):
    start_time = time.time()

    # Directly using the user's question as input for rag_chain.invoke
    response = rag_chain.invoke(user_question)

    # Measure the response time
    end_time = time.time()
    response_time = f"Response time: {end_time - start_time:.2f} seconds."

    # Combine the response and the response time into a single string
    full_response = f"{response}\n\n{response_time}"

    return full_response

# Setup the Gradio interface
iface = gr.Interface(
    fn=process_question,
    inputs=gr.Textbox(lines=2, placeholder="Hi! I am VineBot."),
    outputs=gr.Textbox(),
    title="VineWise 🍇",
    description="Ask your question here....",
)

# Launch the interface
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_transports\default.py", line 72, in map_httpcore_exceptions
    yield
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_transports\default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    r